In [ ]:
# Part 1. Construct the contact map
# Reading and parsing a PDB file
import numpy as np, matplotlib.pyplot as plt

pdb_file = '3nir.pdb'
cutoff = 6  # in Angstrom
CA_dict = {} # Create a dictionary where each atom number and their residue is paired with their position

with open(pdb_file,'r') as pdb:
    for line in pdb:
        values = line.split()
        if values[0] == "ATOM" and values[2] == "CA": # if the atom is a c-alpha atom get its residue and position
            (atom_num, residue, x, y, z) = (values[1], values[3], values[6], values[7], values[8])
            CA_dict.update({(atom_num, residue): [float(x), float(y), float(z)]})

CM_matrix = np.zeros((len(CA_dict), len(CA_dict))) # create the contact map matrix with all zeros
i = 0
j = 0

# iterate over the atom dictionary to find those atoms which are in contact (within our 6 Angstrom threshold) using
# euclidian distance

for k1, v1 in CA_dict.items():
    for k2, v2 in CA_dict.items():
        # calculate the euclidian distance between two atoms and if below the cutoff change the CM_matrix value to 1
        d = ((v2[0] - v1[0])**2 + (v2[1] - v1[1])**2 + (v2[2] - v1[2])**2)**0.5
        if d < cutoff:
            CM_matrix[i, j] = 1
        j +=1
    i += 1
    j = 0

fig, ax = plt.subplots()
ax.matshow(CM_matrix, cmap=plt.cm.Greys)
plt.show()

# Part 2. Identifying the densest region

i = 0
j = 0
max_density = 0
window_size = 20
alpha_count = 0
beta_count = 0

# Create a sliding window that checks for the densest region of our contact map within a specific window size
while i + window_size <= len(CM_matrix): # Make sure we don't exceed our indexing length
    while j + window_size <= len(CM_matrix):
        # Get indexes of the values in the range of our window size
        indxs = np.ix_([x for x in range(i, i + window_size)], [y for y in range(j, j + window_size)])
        window = CM_matrix[indxs]

        # In our contact matrix, count the non-zero spaces which correspond to contact
        if np.count_nonzero(window) >= max_density:
            max_density = np.count_nonzero(window)
            max_window = window
            AA_indx = [x for x in range(i, i + window_size)]
        j += 1
    i += 1
    j = 0


for m in range(0, len(max_window)):
    for n in range(0, len(max_window)):
        if n >= m and m != n and max_window[m][n] == 1:
            if max_window[m][n - 1] == 1 and max_window[m + 1][n] == 1:
                alpha_count += 1
            else:
                beta_count += 1

AA = [list(CA_dict)[i][1] for i in AA_indx]

print(max_window)
print("The corresponding amino acids are:")
print(*AA)
print("The number of amino acids in the alpha helix: {}".format(alpha_count))
print("The number of amino acids in the beta sheet: {}".format(beta_count))


# Part 3. Counting dense patterns

threshold_density = 5
window_size2 = 5
dict_freq = {}
# Similar to before, but now we count the frequency of the densest regions and print out their count
while i + window_size2 <= len(CM_matrix):
    while j + window_size2 <= len(CM_matrix):
        indxs = np.ix_([x for x in range(i, i + window_size2)], [y for y in range(j, j + window_size2)])
        window2 = CM_matrix[indxs]

        if np.count_nonzero(window2) >= threshold_density:
            if np.array2string(window2) not in dict_freq:
                dict_freq.update({np.array2string(window2): 1})

            else:
                freq = dict_freq.get(np.array2string(window2)) + 1
                dict_freq.update({np.array2string(window2): freq})

        j += 1
    i += 1
    j = 0

for k, v in dict_freq.items():
    print("{}\tFrequency: {}".format(k, v))
